# pip install this
## 0 to PyPI in 60 minutes
### Introduction
It's a great time to be a developer! Great languages, great tools, great documentation, and it's all so easy to get to!

Python is an opinionated language. I'm an opinionated developer. This is fine, as long as it is understood.

I prefer Python 3. This talk is in python 3. That's all I have to say about that

Holy wars are fought over dogma, not philosophy. So, start with [The Zen of Python]( https://www.python.org/dev/peps/pep-0020/)

Syntax checking is great! Takes the thinking out of code formatting. So, I like to use vanilla pep8 via out-of-the-box [flake8](https://pypi.python.org/pypi/flake8)

Python is generally distributed in packages on [PyPI](https://pypi.python.org/pypi)

There is a lot of philosophy and discussion about packaging. We're not going to go into this here, but it is interesting! Instead, this talk is intended to be a pragmatic guide to getting your code published.

https://glyph.twistedmatrix.com/2016/08/python-packaging.html
https://pythonrants.wordpress.com/2013/12/06/why-i-hate-virtualenv-and-pip/

I've focused on simple and easy tools and procedures. Some of the things I talk about will be personal best practices. I'll call these out with _italics_. Your mileage may vary, and I understand and hope you have your own set of best-practices or preferences. I'm happy to talk about them later.

### Starting from not quite 0
At comScore, we use Hipchat for team communication. I wanted to see critical errors in my team's hipchat channel (I'm lead dev...I get to decide that =P)

No plugins available, so let's roll our own using the very cool [Hipchat API](https://www.hipchat.com/docs/apiv2)

Basic package structure should look something like this:
```
    <project dir>/  # Name doesn't actually matter
        |
        |--> setup.py      # Info needed to build your package
        |--> <module dir> # Will be the name you import
                |
                |--> __init__.py  # Can be empty, or can be code. _Don't put metadata in this_
                |--> <module>.py  # Things you want to import *from* your package 
                |--> exec/        # _Where I (beginning very recently) prefer to put executables
                        |
                        |--> <executable>.py
```

In [1]:
import os

In [2]:
import sh  # _This package is sooo cool. Check it out: https://amoffat.github.io/sh/

In [3]:
os.makedirs('../hippy-chat/hippy_chat')  # package will be importable as hippy_chat

In [4]:
with open('handler.py') as source_file:
    print(source_file.read())

import arrow
import buzz
import json
import logging
import requests
import textwrap
import urllib


class HipchatHandler(logging.Handler):
    """
    This module provides a logging handler that sends notifications to a
    hipchat room. The notifications will appear as expandable 'cards'
    with relevant data.

    .. todo:: Add sentry support for the card urls
    """

    def __init__(
        self, url, room, token, *args,
        title='event occured in application',
        **kwargs
    ):
        """
        Initializes the handler

        :param: url:   The base URL to the hipchat server
        :param: room:  The name of the room in which to make the notification
        :param: token: The auth token for the room
        :param: title: An optional title to include in the notification
        """
        self.url = '{url}/v2/room/{room}/notification'.format(
            url=url,
            room=urllib.parse.quote(room),
        )
        self.headers = {
            "Content

In [5]:
sh.cp('handler.py', '../hippy-chat/hippy_chat')

In [6]:
with open('demo.py') as demo_file:
    print(demo_file.read())

import click

import hippy_chat.handler


@click.command()
@click.option('-c', '--config-file', help="the file containing hipchat deets")
@click.option(
    '-l', '--log-level',
    default='ERROR',
    type=click.Choice(['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'])
    help="select the level of messges that should be logged to the room",
)
@click.option(
    '-i', '--message-interval',
    default=5,
    type=int,
    help="How often to message the room",
)
def main(config_file, log_level):

    log_level = getattr(logging, log_level)

    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    with open(os.path.expanduser('~/.hipchat.json')) as config_file:
        config = json.load(config_file)

    url = config['default_url']
    room = config['urls'][url]['default_room']
    token = config['urls'][url][room]['token']

    formatter = logging.Formatter(textwrap.dedent("""
        Severity: %(levelname)s
        Module:   %(module)s [%(pathname)s:%(lineno)d

In [8]:
os.makedirs('../hippy-chat/hippy_chat/exec')

In [9]:
sh.cp('demo.py', '../hippy-chat/hippy_chat/exec/demo.py')